# Practical 3

Aim: Study and cleaning of dataset

## Description of Dataset:
The dataset contains the crawled articles from the internet from around 87 websites for the year 2017
it has url of the page,Source of the website,crawl time , title of the news, summary and trimmed descriptions
it has 4 .csv partitions which aggregates to over 1gb
Since there is no character encoding on the dataset , each partition is first converted to excel files in UTF-8
this results in reduction of filesize by the factor of 3

Features:
- **url**	: it is the URL of the article from which the data is crawled
- **source**	:Name of the newspaper
- **crawl_time**	:Timestamp of crawling
- **title**: Title of the news
- **trimmed_description**	: A trimmed description of the summary, sometimes this is bigger than the summar itself
- **summary** : summary of the news, also our main focus is on this column

Further study of the dataset is done while creating a crawler for the dataset, This approach gave me more insight to the following questions:
- Is the dataset bias in some category? : No, the type of news for eg: sports,health,government are very much spreaded and the dataset is very well formed

**Why Python?**: The choice of choosing the programming language is completely based on the general benefits of the language,we have R which is a very well made language especially used for Data Science, And we have Python which is used for almost everything and still has a reasonable opensource support for implementing Data Science models.<br>
Python does more than R in other fields but it has its own trade off for that kindof accessibility<br>
and Since this was an introductory course, choosing Python over R creates more opportunities and keep all other options open for me ie not only data science,Ofcourse R is a much better platform and it is essential to know both since every dataset has its own fabric, Using both of the tool in combination when appropriate will give you a much better result

## Cleaning the data and loading necessary modules

## 1.loading necessary modules

In [3]:
%matplotlib inline
# pandas for data manipulation
import pandas as pd
pd.options.mode.chained_assignment = None
# nltk for nlp
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
# list of stopwords like articles, preposition
stop = set(stopwords.words('english'))
from string import punctuation
from collections import Counter
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.tag import StanfordNERTagger
import datefinder,datetime
st = StanfordNERTagger('C:\\Users\windows\\Downloads\\Compressed\\stanford-ner-2018-02-27\\stanford-ner-2018-02-27\\classifiers\\english.all.3class.distsim.crf.ser.gz',
					   'C:\\Users\\windows\\Downloads\\Compressed\\stanford-ner-2018-02-27\\stanford-ner-2018-02-27\\stanford-ner.jar',
					   encoding='utf-8')
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import matplotlib.pyplot as plt
import numpy as np

### loading the dataset:
The dataset has been divided into 4 parts,document is focused only on the first 1Lac records of first part at the moment
due to memory and cpu constraints

In [4]:
# reading from a excel ,skipfooter defines how many rows you want to leave
df=pd.read_excel(r'C:\Users\windows\Desktop\Bigdata dataset\1.xlsx',skipfooter=350000)

In [5]:
#df2=pd.read_excel(r'C:\Users\windows\Desktop\Bigdata dataset\2.xlsx',skipfooter=400000)

In [6]:
#df3=pd.read_excel(r'C:\Users\windows\Desktop\Bigdata dataset\3.xlsx',skipfooter=279000)

In [7]:
#df4=pd.read_excel(r'C:\Users\windows\Desktop\Bigdata dataset\4.xlsx',skipfooter=210000)

Information about the cities has been downloaded from the internet for retrieving and removal of cities

In [8]:
cities=pd.read_excel(r'C:\Users\windows\Desktop\Bigdata dataset\Cities.xls')
cities.columns
cities=cities.dropna()
citylist=cities['Name of City'].values
statelist=set(cities['State'].values)

In [9]:
## Appending all the above dataframes into
# dfl=[]
# dfl.append(df1)
# dfl.append(df2)
# dfl.append(df3)
# dfl.append(df4)
# df=pd.concat(dfl,ignore_index=True)

In [10]:
df.shape ## here we have the size of the data we are going to work upon

(100281, 6)

In [11]:
df.head() ## top 5 records to see how the data looks

,url,source,crawl_time,title,trimmed_description,summary
0,http://www.thehindu.com/news/national/New-Army...,The Hindu,2017-01-01T00:19:03Z,"New Army, Air Force Chiefs assume charge",The Army and the Air Force got new chiefs on S...,The Army and the Air Force got new chiefs on S...
1,http://www.thehindu.com/news/national/kerala/K...,The Hindu,2017-01-01T00:19:03Z,KSRTC unions call for stir on Jan. 4,The Indian National Trade Union Congress (INTU...,"Utility needs ₹174.5 crore to pay salary, two ..."
2,http://www.thehindu.com/news/cities/mumbai/Luc...,The Hindu,2017-01-01T00:19:03Z,Lucky escape for tourists in SGNP bus mishap,Mumbai: More than 30 people on board a bus ope...,Mumbai: More than 30 people on board a bus ope...
3,http://www.thehindu.com/news/cities/Hyderabad/...,The Hindu,2017-01-01T00:19:03Z,2016: A successful year for Tollywood,The year just gone by will undoubtedly go down...,The year saw a diverse range of movies being r...
4,http://www.sify.com/news/china-to-ban-ivory-tr...,Sify,2017-01-01T00:19:04Z,China to ban ivory trade by end of 2017,php //if(function_exists('ismobile') && !ismob...,Beijing: China will ban the processing and sal...


Title , trimmed_description and summary contains the title of the newspaper which will harm the analysis and hence it has removed

#### Task 1:Remove the names of newspapers from the title and summary

In [12]:
newspaper_df=df['source'] ##storing the source column which has the names of the newspaper

In [13]:
newspaper_df=newspaper_df.drop_duplicates() ## removing duplicates 

In [14]:
newsnamelist=newspaper_df.values ## dropping null values

In [ ]:
newsnamelist ## this list contains the names

#### Datatypes
since the data is imported in Dataframes,python will try to assign datatypes on its own and this is where the confusion gets created, You will get "expected buffer or string type" when the dtypes are not consistent and hence for ease of operation,its better to specify the dtypes before and do the cleaning of null values before performing an transformation on the data

In [16]:
##specifying the dtypes of the columns
df['url']=df['url'].astype(str)
df['title']=df['title'].astype(str)
df['trimmed_description']=df['trimmed_description'].astype(str)
df['summary']=df['summary'].astype(str)
df['source']=df['source'].astype(str)

In [17]:
df.shape

(100281, 6)

In [18]:
df['title']=df['title'].drop_duplicates()

In [19]:
df['summary']=df['summary'].drop_duplicates()

In [20]:
df['title']=df['title'].dropna()

In [21]:
df['summary']=df['summary'].dropna()

In [22]:
df['trimmed_description']=df['trimmed_description'].drop_duplicates()

In [23]:
df['trimmed_description']=df['trimmed_description'].dropna()

In [24]:
## change to string
def toString(text):
    text=str(text)
    return text

In [25]:
df['title']=df['title'].apply(lambda x:toString(x))

In [26]:
df['summary']=df['summary'].apply(lambda x:toString(x))

In [27]:
df['trimmed_description']=df['trimmed_description'].apply(lambda x:toString(x))

In [28]:
df['source']=df['source'].apply(lambda x:toString(x))

In [29]:
df['crawl_time']=df['crawl_time'].apply(lambda x:toString(x))

In [30]:
df=df[df.source!='nan']

In [31]:
df=df[df.title!='nan']

In [32]:
df=df[df.trimmed_description!='nan']

In [33]:
df=df[df.url!='nan']

In [34]:
df=df[df.summary!='nan']

In [35]:
df=df[df.source!='NaN']

In [36]:
df=df[df.title!='NaN']

In [37]:
df=df[df.trimmed_description!='NaN']

In [38]:
df=df[df.url!='NaN']

In [39]:
df=df[df.summary!='NaN']

In [40]:
df.shape

(68557, 6)

In [41]:
df.dtypes


url                    object
source                 object
crawl_time             object
title                  object
trimmed_description    object
summary                object
dtype: object